In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
import os
from pinecone import Pinecone, ServerlessSpec
from transformers import pipeline
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [3]:
load_dotenv()

True

In [4]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
if not PINECONE_API_KEY:
    raise ValueError("Pinecone API key is missing. Please set the PINECONE_API_KEY in the .env file.")

In [5]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [6]:
index_name = "medical-chatbot"
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

/tmp/ipykernel_54051/3125231043.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")


In [7]:
get_op = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
    )

In [8]:
get_op

In [9]:
retriever = get_op.as_retriever(search_type="similarity", search_kwards={"k":3})

In [10]:
llm = OllamaLLM(model="llama3.1", temperature=0.4, max_tokens=500)

In [22]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know and end the response dont say anything else. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [23]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [24]:
query = "What is diabetes?"
response = rag_chain.invoke({"input": query})
print(f"'{query}': \
      \n\n {response['answer']}")

'What is diabetes?':       

 Diabetes mellitus is a chronic disease where the pancreas doesn't produce enough insulin or cells don't respond to it, causing high blood sugar levels. Symptoms include frequent urination, lethargy, excessive thirst, and hunger. It can lead to serious health complications like kidney failure, heart disease, stroke, and blindness.


In [25]:
query = "What is linear algebra?"
response = rag_chain.invoke({"input": query})
print(f"'{query}': \
      \n\n {response['answer']}")

'What is linear algebra?':       

 I don't know.
